In [1]:
from docplex.mp.model import Model

def sucesso():
    print("OK")
    
sucesso()

OK


In [2]:
model = Model(name = "Modelo DLMPF")

# Número de linhas, itens, períodos e recursos

P = 1
R = 2
T = 2 
L = 2

CONJ_P_itens = list(range(1,P+1))
CONJ_R_rec = list(range(1,R+1))
CONJ_T_per = list(range(0,T+1))
CONJ_L_lin = list(range(1,L+1))

CONJ_Pr_it_rec = CONJ_P_itens #por hora, mudar isso depois!
CONJ_Li_lin_it = CONJ_L_lin #por hora, mudar isso depois!

print(CONJ_P_itens, CONJ_R_rec, CONJ_T_per, CONJ_L_lin, CONJ_Pr_it_rec, CONJ_Li_lin_it)


sucesso()

[1] [1, 2] [0, 1, 2] [1, 2] [1] [1, 2]
OK


In [3]:
# Capacidades

Cr = [1200, 600]
Cd = [600, 300]
Ch = [[6, 3],[6,6]]
p = [[5,2], [3,4]]
        
print(Cr, Cd, Ch, p)
sucesso()

[1200, 600] [600, 300] [[6, 3], [6, 6]] [[5, 2], [3, 4]]
OK


In [4]:
# Demandas

Q = 240
q = 20
d = [480]
d_num_lotes = [2]
d_tam_residual = [0]  

# Penalidade

H = 0

print(Q, q, d, d_num_lotes, d_tam_residual, H)
sucesso()

240 20 [480] [2] [0] 0
OK


As variáveis de decisão são: xikt, yikt, zikt e wikt, sendo: 

x uma variável inteira que representa o número de lotes de tamanho Q
y uma variável binária que indica se houver produção do residual de demanda d
z uma variável binária que indica se houver fracionamento do lote do tipo de item i
w uma variável interia da quantidade do tipo de item i na linha k

In [5]:
# Variáveis de decisão

x = {}
y = {}
z = {}
w = {}

x = model.integer_var_cube(keys1=range(1,P+1), keys2=range(1,L+1), keys3=range(0,T+1), name='X')
y = model.binary_var_cube(keys1=range(1,P+1), keys2=range(1,L+1), keys3=range(0,T+1), name='Y')
w = model.integer_var_cube(keys1=range(1,P+1), keys2=range(1,L+1), keys3=range(0,T+1), name='W')
z = model.binary_var_cube(keys1=range(1,P+1), keys2=range(1,L+1), keys3=range(0,T+1), name='Z')

sucesso()

OK


In [6]:
# Restrição 2
for i in CONJ_P_itens:
    model.add_constraint(model.sum(Q * x[i,k,t] + d_num_lotes[i-1] * y[i,k,t] for k in CONJ_L_lin for t in CONJ_T_per) == d[i-1], ctname = 'R2')

# Restrição 3

for i in CONJ_P_itens:
    model.add_constraint(model.sum(x[i,k,t] for k in range(1,L+1) for t in range(T+1)) == d_tam_residual[i-1], ctname = 'R3')
    
# Restrição 4

for i in CONJ_P_itens:
    model.add_constraint(model.sum(y[i, k, t] for k in range(1,L+1) for t in range(T+1)) <= 1, ctname = 'R4')

# Restrição 5

for i in CONJ_P_itens:
    for k in CONJ_L_lin:
        for t in CONJ_T_per:
            model.add_constraint(w[i, k, t] <= Q * x[i, k, t] + d_num_lotes[i-1] * y[i, k, t], ctname = 'R5')

# Restrição 6

for i in CONJ_P_itens:
    for k in CONJ_L_lin:
        for t in CONJ_T_per:
            model.add_constraint(w[i, k, t] <= (Q - q) * z[i,k,t], ctname = 'R6')
            
# Restrição 7

for i in CONJ_P_itens:
    for k in CONJ_L_lin:
        for t in CONJ_T_per:
            model.add_constraint(q * z[i,k,t] <= w[i,k,t], ctname = 'R7')
            
# Restrição 8
for k in CONJ_L_lin:
        for t in CONJ_T_per:
            model.add_constraint(model.sum(z[i,k,t] for i in CONJ_P_itens)<=1, ctname = 'R8')
            
# Restrição 9

for r in CONJ_R_rec:
    model.add_constraint(model.sum(Q * x[i,k,t] + d_num_lotes[i-1] * y[i,k,t] for i in range(1,P+1) for k in range(1,L+1) for t in range(T+1)) <= Cr[r-1], ctname = "R9")

# Restrição 10

for r in CONJ_R_rec:
    for t in CONJ_T_per:
        if t != 0:
            model.add_constraint(model.sum(Q * x[i,k,t] + d_num_lotes[i-1] * y[i,k,t] + w[i, k, t-1] - w[i,k,t] for i in range (1,P+1) for k in range(1,L+1)) <= Cd[r-1], ctname = "R1-")
        else:
            model.add_constraint(model.sum(Q * x[i,k,t] + d_num_lotes[i-1] * y[i,k,t] + 0 - w[i,k,t] for i in range (1,P+1) for k in range(1,L+1)) <= Cd[r-1], ctname = "R10")

# Restrição 11

for k in CONJ_L_lin:
    for t in CONJ_T_per:
        if t!=0:
            model.add_constraint(model.sum(p[i-1][k-1] * (Q * x[i,k,t] + (d[i-1] % Q) * y[i,k,t] + w[i,k,t-1] - w[i,k,t]) for i in CONJ_P_itens) <= Ch[k-1][t-1], ctname = 'R11')

# Restrição 12

for i in CONJ_P_itens:
    for k in CONJ_L_lin:
        model.add_constraint(w[i,k,0] == 0, ctname = 'R12')
        
# Restrição 13
        
for i in CONJ_P_itens:
    for k in CONJ_L_lin:
        model.add_constraint(w[i,k,T] == 0, ctname = 'R13')

        
#Restrição 14
for i in CONJ_P_itens:
    for k in CONJ_L_lin:
        for t in CONJ_T_per:
            model.add_constraint(0<=w[i,k,t], ctname = 'R14.1')
            model.add_constraint(w[i,k,t]<= Q - q, ctname = 'R14.2')

            
sucesso()

OK


In [7]:
objetivo = model.sum(
    p[i][k] * Q * x[i, k, t] + p[i][k] * (d[i-1] % Q) * y[i, k, t] + z[i, k, t] 
    for i in CONJ_P_itens 
    for k in CONJ_L_lin 
    for t in CONJ_T_per 
    if t > 0 and i in p and k in p[i])

model.minimize(objetivo)

sucesso()

OK


In [8]:
#Perguntar somatório função objetivo
#Perguntar listas x dicionários
#Perguntar uns tipo tempo de processamento

In [9]:
print(model.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Modelo DLMPF

Minimize
 obj:
Subject To
 R2: 240 X_1_1_0 + 240 X_1_1_1 + 240 X_1_1_2 + 240 X_1_2_0 + 240 X_1_2_1
     + 240 X_1_2_2 + 2 Y_1_1_0 + 2 Y_1_1_1 + 2 Y_1_1_2 + 2 Y_1_2_0 + 2 Y_1_2_1
     + 2 Y_1_2_2 = 480
 R3: X_1_1_0 + X_1_1_1 + X_1_1_2 + X_1_2_0 + X_1_2_1 + X_1_2_2 = 0
 R4: Y_1_1_0 + Y_1_1_1 + Y_1_1_2 + Y_1_2_0 + Y_1_2_1 + Y_1_2_2 <= 1
 R5: W_1_1_0 - 240 X_1_1_0 - 2 Y_1_1_0 <= 0
 R5#4: W_1_1_1 - 240 X_1_1_1 - 2 Y_1_1_1 <= 0
 R5#5: W_1_1_2 - 240 X_1_1_2 - 2 Y_1_1_2 <= 0
 R5#6: W_1_2_0 - 240 X_1_2_0 - 2 Y_1_2_0 <= 0
 R5#7: W_1_2_1 - 240 X_1_2_1 - 2 Y_1_2_1 <= 0
 R5#8: W_1_2_2 - 240 X_1_2_2 - 2 Y_1_2_2 <= 0
 R6: W_1_1_0 - 220 Z_1_1_0 <= 0
 R6#10: W_1_1_1 - 220 Z_1_1_1 <= 0
 R6#11: W_1_1_2 - 220 Z_1_1_2 <= 0
 R6#12: W_1_2_0 - 220 Z_1_2_0 <= 0
 R6#13: W_1_2_1 - 220 Z_1_2_1 <= 0
 R6#14: W_1_2_2 - 220 Z_1_2_2 <= 0
 R7: 20 Z_1_1_0 - W_1_1_0 <= 0
 R7#16: 20 Z_1_1_1 - W_1_1_1 <= 0
 R7#17: 20 Z_1_1_2 - W_1_

In [11]:
sol = model.solve()
print(sol)

None
